In [ ]:
import gentrl
import torch
from rdkit.Chem import Draw
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol
from IPython.display import SVG, display

# torch.cuda.set_device(0)

In [ ]:
args = {}
args['rank'] = 0

def Net():

    latent_descr = 50 * [('c', 20)]
    feature_descr = [('c', 20)]
    latent_size = 50
    latent_input_size = 50

    enc = gentrl.RNNEncoder(latent_size)
    dec = gentrl.DilConvDecoder(latent_input_size,args)
    model = gentrl.GENTRL(enc, dec, latent_descr,
                              feature_descr, beta=0.001)
    return model

In [ ]:
model = Net()

In [ ]:
model.load('saved_gentrl_after_rl/')
model.cuda();

In [ ]:
def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=True, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [ ]:
generated = []

while len(generated) < 1000:
    sampled = model.sample(100)
    sampled_valid = [s for s in sampled if get_mol(s)]
    
    generated += sampled_valid

In [ ]:
mols = []
legends = []
for s in generated:
    mols.append(get_mol(s))
    legends.append(str(penalized_logP(s)))

result = Draw.MolsToGridImage(mols, legends=legends, useSVG=True)

In [ ]:
display(SVG(result))